In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data - it's a JSON array
train_df = pd.read_json('/home/data/train.json')
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")
print("\nTarget distribution:")
print(train_df['requester_received_pizza'].value_counts(normalize=True))
print("\nFirst few rows:")
train_df.head()